In [1]:
import findspark
findspark.init()

import import_ipynb
from utils import rename

import pyspark.sql.types as tp
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression

importing Jupyter notebook from utils.ipynb


### Classification

In [2]:
sc = SparkContext()

url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
sc.addFile(url)
sqlContext = SQLContext(sc)

In [3]:
schema_setting = tp.StructType([
    tp.StructField(name='sepal.length', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='sepal.width', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='petal.length', dataType=tp.DoubleType(), nullable=True),
    tp.StructField(name='petal.width', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='variety', dataType=tp.StringType(), nullable=False)
])

data = sqlContext.read.csv(
    SparkFiles.get("iris.csv"), 
    schema=schema_setting,
    header=True
)

In [4]:
data.show(3)

+------------+-----------+------------+-----------+-------+
|sepal.length|sepal.width|petal.length|petal.width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
+------------+-----------+------------+-----------+-------+
only showing top 3 rows



In [5]:
rename_data = rename(data, data.columns, ['x1', 'x2', 'x3', 'x4', 'y'])
rename_data.show(3)

+---+---+---+---+------+
| x1| x2| x3| x4|     y|
+---+---+---+---+------+
|5.1|3.5|1.4|0.2|Setosa|
|4.9|3.0|1.4|0.2|Setosa|
|4.7|3.2|1.3|0.2|Setosa|
+---+---+---+---+------+
only showing top 3 rows



In [6]:
split = rename_data.randomSplit([0.6, 0.4])

train_data = split[0]
test_data = split[1]

In [7]:
# string -> category
string_indexer = StringIndexer(inputCol='y', outputCol='label')

# each x value -> vector
vector_assembler = VectorAssembler(inputCols=['x1', 'x2', 'x3', 'x4'], outputCol='features')

# generage model
logistic_regression = LogisticRegression(featuresCol='features', labelCol='label')

# combine
pipeline = Pipeline(stages=[string_indexer, vector_assembler, logistic_regression])

# create model
model = pipeline.fit(train_data)

In [8]:
pred = model.transform(test_data)
pred.select('label', 'prediction').groupBy('label', 'prediction').count().sort('label').show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       0.0|   19|
|  1.0|       0.0|    2|
|  1.0|       1.0|   17|
|  2.0|       2.0|   23|
+-----+----------+-----+



### Regression

In [10]:
sc.stop()
sc = SparkContext()

url = 'https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv'
sc.addFile(url)
sqlContext = SQLContext(sc)

In [11]:
schema_setting = tp.StructType([
    tp.StructField(name='crim', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='zn', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='indus', dataType=tp.DoubleType(), nullable=True),
    tp.StructField(name='chas', dataType=tp.IntegerType(), nullable=False),
    tp.StructField(name='nox', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='rm', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='age', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='dis', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='rad', dataType=tp.IntegerType(), nullable=False),
    tp.StructField(name='tax', dataType=tp.IntegerType(), nullable=False),
    tp.StructField(name='ptratio', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='b', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='lstat', dataType=tp.DoubleType(), nullable=False),
    tp.StructField(name='medv', dataType=tp.DoubleType(), nullable=False)
])

data = sqlContext.read.csv(
    SparkFiles.get("BostonHousing.csv"), 
    schema=schema_setting,
    header=True
)

In [12]:
data.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



In [14]:
split = data.randomSplit([0.6, 0.4])

train_data = split[0]
test_data = split[1]

In [15]:
input_cols = train_data.columns[:-1]

train_vectors = VectorAssembler(inputCols=input_cols, outputCol='features')
train = train_vectors.transform(train_data)

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol='features', labelCol='medv')

lrModel = lr.fit(train)


In [16]:
trainingSummary = lrModel.summary

print("RMSE: %f" % trainingSummary.rootMeanSquaredError)

RMSE: 4.785258


In [17]:
test_vectors = VectorAssembler(inputCols=input_cols, outputCol='features')
test = train_vectors.transform(test_data)

pred = lrModel.transform(test)
pred.select('medv', 'prediction').show(3)

+----+------------------+
|medv|        prediction|
+----+------------------+
|24.0|30.652205857897826|
|32.7|32.239020383843304|
|50.0|41.950905237627715|
+----+------------------+
only showing top 3 rows

